In [1]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'


import findspark
findspark.init()
findspark.find()


'/usr/lib/spark/'

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
spark = SparkSession \
    .builder \
    .master("yarn") \
        .config("spark.driver.cores", "2") \
        .config("spark.driver.memory", "2g") \
        .appName("myapp2") \
        .getOrCreate()

print(spark)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2023-08-08 20:06:27,574 WARN util.Utils: Your hostname, fhmnhmbbc0rb3lsrjbec resolves to a loopback address: 127.0.1.1; using 172.16.0.38 instead (on interface eth0)
2023-08-08 20:06:27,575 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-08-08 20:06:29,478 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update 

In [3]:
import pyspark.sql.functions as F 
events = spark.read.json("/user/master/data/events/date=2022-05-31")
events.printSchema()

root
 |-- event: struct (nullable = true)
 |    |-- admins: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- channel_id: long (nullable = true)
 |    |-- datetime: string (nullable = true)
 |    |-- media: struct (nullable = true)
 |    |    |-- media_type: string (nullable = true)
 |    |    |-- src: string (nullable = true)
 |    |-- message: string (nullable = true)
 |    |-- message_channel_to: long (nullable = true)
 |    |-- message_from: long (nullable = true)
 |    |-- message_group: long (nullable = true)
 |    |-- message_id: long (nullable = true)
 |    |-- message_to: long (nullable = true)
 |    |-- message_ts: string (nullable = true)
 |    |-- subscription_channel: long (nullable = true)
 |    |-- tags: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- user: string (nullable = true)
 |-- event_type: string (nullable = true)



In [5]:
events.withColumn('hour',F.hour(F.col('event.datetime'))) \
      .withColumn('minute', F.minute(F.col('event.datetime'))) \
      .withColumn('second', F.second(F.col('event.datetime')))\
      .orderBy("event.datetime", ascending=False).show(10)

+--------------------+------------+----+------+------+
|               event|  event_type|hour|minute|second|
+--------------------+------------+----+------+------+
|[[19342], 987160,...|     message|  23|    57|    38|
|[,, 2022-05-31 23...|subscription|  23|    56|    58|
|[[26358], 247511,...|     message|  23|    56|    53|
|[[79792], 748847,...|     message|  23|    56|    27|
|[,, 2022-05-31 23...|subscription|  23|    53|    42|
|[,, 2022-05-31 23...|subscription|  23|    53|    14|
|[[151897], 396845...|     message|  23|    52|    15|
|[,, 2022-05-31 23...|subscription|  23|    52|     8|
|[,, 2022-05-31 23...|subscription|  23|    51|    19|
|[,, 2022-05-31 23...|subscription|  23|    51|    15|
+--------------------+------------+----+------+------+
only showing top 10 rows



In [6]:
event_from=events.filter(F.col('event_type')=='message').groupBy(F.col('event.message_from')).count()
event_from.orderBy(F.col('count').desc()).show() 


+------------+-----+
|message_from|count|
+------------+-----+
|       49528|   13|
|       93872|   13|
|      139087|    8|
|       62893|    5|
|      111641|    4|
|      148319|    4|
|       66975|    4|
|       71582|    3|
|      146005|    3|
|       42341|    3|
|      107050|    3|
|      103542|    2|
|      143013|    2|
|       74592|    2|
|       60524|    2|
|       85580|    2|
|      124602|    2|
|      110023|    2|
|      154838|    2|
|       72522|    2|
+------------+-----+
only showing top 20 rows



In [8]:
event_day_max = event_from.select(F.max('count') )
event_day_max.withColumnRenamed('max(count)', 'max_count').show()

+---------+
|max_count|
+---------+
|       13|
+---------+

